# Is arousal well distributed among recommendation batchs?

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# get data
ar_val = pd.read_csv("/home/sacha/Docker/reco_docker_arousal/Recommender/data/arousal_valence.csv")
metadata = pd.read_csv("/home/sacha/Docker/reco_docker_arousal/Recommender/data/metadata.csv")
sim = pd.read_csv("/home/sacha/Docker/reco_docker_arousal/Recommender/data/similarities.csv")

# change arousal to int in order to be able to plot distribution
ar_val['arousal'] = ar_val.arousal.astype(int) 
distrib = ar_val.groupby(['arousal']).agg('size') / ar_val.shape[0]
plt.plot(distrib)
plt.xlabel("arousal")
plt.ylabel("proportion")
plt.title("arousal distribution in dataset")
plt.show()

# We define 3 scales of arousal :
### From 0 to 53 : Chill songs

In [16]:
pd.set_option('display.max_colwidth', -1)

chillT = pd.merge(ar_val.query("arousal >= 0 and arousal < 53"),metadata, on="id")
print(str(chillT.shape[0]) + " chill tracks")

chillSample = chillT.sample(4)
chillSample[["artist","track","arousal","preview"]]

25035 chill tracks


,artist,track,arousal,preview
23343,Peter Von Poehl,Tooth Fairy,42,https://e-cdns-preview-2.dzcdn.net/stream/28c90927299374c775c60ea547ee7ebd-2.mp3
17316,Local Natives,Breakers,51,https://e-cdns-preview-0.dzcdn.net/stream/0c6167d2dc2445322c08e28c56849c6a-2.mp3
7238,L5,Te Garder Près De Moi,52,https://e-cdns-preview-d.dzcdn.net/stream/d7192dade94cbb816aa8fdb0fba3bc07-4.mp3
16644,Ludovico Einaudi & Ballaké Sissoko,Laissez Moi En Paix,36,https://e-cdns-preview-6.dzcdn.net/stream/6f05265354bf4d51ddc3c06daaf67e6d-1.mp3


### From 53 to 64 : Medium songs

In [15]:
mediumT = pd.merge(ar_val.query("arousal >= 53 and arousal < 64"),metadata, on="id")
print(str(mediumT.shape[0]) + " medium tracks")

mediumSample = mediumT.sample(4)
mediumSample[["artist","track","arousal","preview"]]

28605 medium tracks


,artist,track,arousal,preview
26042,Patrice,Million Miles (Album Version),61,https://e-cdns-preview-e.dzcdn.net/stream/efd6aefb1c16ac99ad5f8f6e1c2aa5ea-5.mp3
21944,Thievery Corporation,The Richest Man in Babylon,56,https://e-cdns-preview-e.dzcdn.net/stream/e76f2f7f1a2a9752602413e69300e45f-2.mp3
2503,Guns N' Roses,Knockin' On Heaven's Door (Live In London / 1992),57,https://e-cdns-preview-d.dzcdn.net/stream/d3c1ca970717c2d62c48a154d99f9d00-4.mp3
6424,Aminé,REDMERCEDES,62,https://e-cdns-preview-5.dzcdn.net/stream/5832fdd66f63133414845ca5a98c3ab4-4.mp3


### From 65 to 100 : Hardcore songs

In [17]:
hardcoreT = pd.merge(ar_val.query("arousal >= 64 and arousal < 100"),metadata, on="id")
print(str(hardcoreT.shape[0]) + " hardcore tracks")

hardcoreSample = hardcoreT.sample(4)
hardcoreSample[["artist","track","arousal","preview"]]

35314 hardcore tracks


,artist,track,arousal,preview
15108,The Kooks,Always Where I Need to Be,74,https://e-cdns-preview-e.dzcdn.net/stream/e6bdabdb910936f52a9ae01ad8b7a118-7.mp3
32119,Johnny Marr,Easy Money,75,https://e-cdns-preview-4.dzcdn.net/stream/42ca19fd7730767b6094504e95eb710e-0.mp3
24979,Jupiter,One O Six (The Supermen Lovers Classic Remix),70,https://e-cdns-preview-c.dzcdn.net/stream/cef14b30694374a67870b1c2f85e5692-0.mp3
25913,Mathieu Bouthier,New York City (Original Radio Edit - French Version),68,https://e-cdns-preview-6.dzcdn.net/stream/6680cb08205695f2696e40d2d75341cf-3.mp3


# Probability to have a batch of recommendation that contains tracks in all the scales.
=> We compute the mean and standard dev of the arousals in all the batches. Then we compute the mean of the mean arousals and the mean of the standard devs. Then we know that we have a probability of 95% to fall between these values.

In [96]:
### import numpy as np

# we convert our recommendation matrix to 'long' format.
simLong = pd.DataFrame(np.array(sim).flatten(),columns = ['id'])

# we labelise each batch of recommendation
simLong['label'] = [el // 31 for el in list(range(0,len(simLong)))]

# we add the arousal of the tracks
merged = pd.merge(simLong,ar_val, on = ["id"])

# we group by batch and compute arousal mean and stddev for each batch
arousalRanges = merged.groupby("label").agg({'arousal':[np.min,np.max]})
arousalRanges["min"] = arousalRanges.arousal.amin
arousalRanges["max"] = arousalRanges.arousal.amax
mins = arousalRanges['min'] <= 53
maxs = arousalRanges['max'] >= 64
c = mins & maxs
x = c.value_counts()[1]
y = c.value_counts()[0]

print("proportion of batches that possess tracks in all scales : " + str(x / (x+y)))

proportion of batches that possess tracks in all scales : 0.7870057488919121
